In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

# import sweetviz as sv 

from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import make_scorer, roc_auc_score
from sklearn.metrics import accuracy_score


import plotly.express as px
import plotly.graph_objects as go

import torch
import torch.nn as nn
import torch.optim as optim

from IPython.display import Image, display
from tqdm import tqdm
import imageio
import os

import warnings
# warnings.filterwarnings("ignore")

device = torch.device("cpu")

ModuleNotFoundError: No module named 'torch'

In [ ]:
w